In [1]:
# need to translate the GTK / Luke sitetypes into classes
# should we just adjust SpaFHy parameters to be directly compatible with GTK classification?
# if we use sitetypes as in Hyytiälä, then we should make the routine similar as there..
# saving to .asc files (topsoil, rootsoil, deepsoil)

In [2]:
import rasterio
import numpy as np
import tools
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
s_soil_file = r'/Users/jpnousu/WBT_data/rasters/soil_16/surface200.asc'
b_soil_file = r'/Users/jpnousu/WBT_data/rasters/soil_16/bottom200.asc'
mt_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/paatyyppi_vmi1x_1721.asc'
sf_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/kasvupaikka_vmi1x_1721.asc'
lk_file = r'/Users/jpnousu/WBT_data/rasters/maastotietokanta_16/jarvi.asc'

# --- Surface soil ---
with rasterio.open(s_soil_file) as src:
    s_soil = src.read(1)
    s_soil_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    s_soil = np.where(s_soil == old_nodata, -9999, s_soil)
#s_soil = tools.interpolate_nodata(s_soil, nodata=-9999)
s_soil_meta.update({"nodata": -9999, "dtype": "int32"})

# --- Bottom soil ---
with rasterio.open(b_soil_file) as src:
    b_soil = src.read(1)
    b_soil_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    b_soil = np.where(b_soil == old_nodata, -9999, b_soil)
#b_soil = tools.interpolate_nodata(b_soil, nodata=-9999)
b_soil_meta.update({"nodata": -9999, "dtype": "int32"})

# --- Main type ---
with rasterio.open(mt_file) as src:
    mt = src.read(1)
    mt_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    mt = np.where(mt == old_nodata, -9999, mt)
#mt = tools.interpolate_nodata(mt, nodata=-9999)
mt_meta.update({"nodata": -9999, "dtype": "int32"})

# --- Site fertility ---
with rasterio.open(sf_file) as src:
    sf = src.read(1)
    sf_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    sf = np.where(sf == old_nodata, -9999, sf)
#sf = tools.interpolate_nodata(sf, nodata=-9999)

# --- Lake mask ---
with rasterio.open(lk_file) as src:
    lk = src.read(1)
    lk_meta = src.meta.copy()
    old_nodata = src.nodata

In [4]:
# manipulating surface and bottom soils from GTK
CoarseTextured = [195213, 195314, 19531421, 195313, 195310]
MediumTextured = [195315, 19531521, 195215, 195214, 195601, 195411, 195112,
                  195311, 195113, 195111, 195210, 195110, 195312]
FineTextured = [19531521, 195412, 19541221, 195511, 195413, 195410,
                19541321, 195618]
Peats = [195512, 195513, 195514, 19551822, 19551891, 19551892]
Water = [195603]

# Combine into a dictionary for easy looping
soil_map = {
    1: CoarseTextured,
    2: MediumTextured,
    3: FineTextured,
    4: Peats,
    5: Water
}

# Reclassify soils
s_soil_classified = tools.reclassify_soil(s_soil, soil_map)
b_soil_classified = tools.reclassify_soil(b_soil, soil_map)

# Save
out_s_soil_file = "/Users/jpnousu/WBT_data/rasters/soil_16/top_soil_gtk.asc"
with rasterio.open(out_s_soil_file, "w", **s_soil_meta) as dst:
    dst.write(s_soil_classified, 1)

out_b_soil_file = "/Users/jpnousu/WBT_data/rasters/soil_16/bottom_soil_gtk.asc"
with rasterio.open(out_b_soil_file, "w", **b_soil_meta) as dst:
    dst.write(b_soil_classified, 1)

In [5]:
out_mt_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/maintype_mnfi.asc"
mt_new = mt.copy()
mt_new[mt_new == -9999] = 5 # 'nodata'
with rasterio.open(out_mt_file, "w", **mt_meta) as dst:
    dst.write(mt_new, 1)

In [6]:
maintype_binary = mt.copy()
# 1 = mineral, 2 = peatland (or water)
maintype_binary[np.isin(maintype_binary, [2,3,4])] = 2
maintype_binary[maintype_binary == 1] = 1

# Prepare output
sitetype_comb = np.zeros_like(sf, dtype=np.int32)

# Loop only over unique site types
for stype in np.unique(sf):
    mask_forest = (maintype_binary == 1) & (sf == stype)
    mask_mire = (maintype_binary == 2) & (sf == stype)
    
    sitetype_comb[mask_forest] = stype
    sitetype_comb[mask_mire] = stype + 10

sitetype_comb[sitetype_comb == 0] = 17

# Save as .asc
out_sitetype_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/sitefertility_mnfi.asc"
with rasterio.open(out_sitetype_file, "w", **sf_meta) as dst:
    dst.write(sitetype_comb, 1)